In [2]:
!pip install pygame


distutils: C:\Users\Elssky\anaconda3\envs\tensorflow2\Include\UNKNOWN
sysconfig: C:\Users\Elssky\anaconda3\envs\tensorflow2\Include
user = False
home = None
root = None
prefix = None
distutils: C:\Users\Elssky\anaconda3\envs\tensorflow2\Include\UNKNOWN
sysconfig: C:\Users\Elssky\anaconda3\envs\tensorflow2\Include
user = False
home = None
root = None
prefix = None


In [2]:
from music21 import converter, instrument, note, chord, stream

def get_notes(song_path,song_names):
    """获得midi音乐文件中的音符

    :param song_path: [文件的保存地址]
    :type song_path: [str]
    :param song_names: [所有音乐文件的文件名]
    :type song_names: [list]
    :return: [所有符合要求的音符]
    :rtype: [list]
    """
    all_notes = []
    for song_name in song_names:
        stream = converter.parse(song_path+song_name)
        instru = instrument.partitionByInstrument(stream)
        if instru:  # 如果有乐器部分，取第一个乐器部分
            notes = instru.parts[0].recurse()
        else:  #如果没有乐器部分，直接取note
            notes = stream.flat.notes
        for element in notes:
            # 如果是 Note 类型，取音调
            # 如果是 Chord 类型，取音调的序号,存int类型比较容易处理
            if isinstance(element, note.Note):
                all_notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                all_notes.append('.'.join(str(n) for n in element.normalOrder))
    return all_notes

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


In [3]:
def save_data(filename,content):
    """保存音符

    :param filename: [保存的文件名]
    :type filename: [str]
    :param content: [内容]
    :type content: [list]]
    """
    with open(filename,"w") as f:
        for data in content:
            f.write(str(data)+"\n")

def get_data(filename):
    """从文件中获取音符

    :param filename: [文件名]
    :type filename: [str]
    :return: [返回音符]
    :rtype: [list]
    """
    with open(filename) as f:
       all_notes = f.readlines()
    return [ note[:len(note)-1]  for note in all_notes]

In [5]:
import os
song_path = "./midi_songs/"
song_names = os.listdir(song_path)

In [6]:
# 获取note数组
all_notes = get_notes(song_path,song_names)
# 保存文件
save_data("data.txt",all_notes)

In [7]:
from collections import Counter
# 对出现过的note进行统计
counter = Counter(all_notes)
# 根据出现的次数，进行从大到小的排序
note_count = sorted(counter.items(),key=lambda x : -x[1])
notes,_ = zip(*note_count)
# 产生note到id的映射
note_to_id = {note:id for id,note in enumerate(notes)}

In [9]:
print(list(note_to_id.items())[:20])

[('A2', 0), ('E3', 1), ('E4', 2), ('9.0.4', 3), ('F#3', 4), ('B3', 5), ('B2', 6), ('A3', 7), ('C5', 8), ('E5', 9), ('9.0', 10), ('F3', 11), ('E2', 12), ('A4', 13), ('C3', 14), ('G3', 15), ('F#5', 16), ('0.4', 17), ('C6', 18), ('D5', 19)]


In [10]:
X_train = []
Y_train = []
sequence_batch = 100
for i in range(len(all_notes)-sequence_batch):
    X_pre = all_notes[i:i+sequence_batch]
    Y_pre = all_notes[i+sequence_batch]
    X_train.append([note_to_id[note] for note in X_pre])
    Y_train.append(note_to_id[Y_pre])

In [15]:
print(len(X_train))

[[18, 7, 2, 18, 31, 2, 18, 7, 2, 18, 31, 2, 9, 7, 2, 9, 32, 2, 9, 7, 2, 9, 32, 2, 16, 7, 2, 16, 13, 2, 16, 7, 2, 16, 13, 2, 8, 7, 2, 8, 36, 2, 8, 7, 2, 8, 36, 2, 18, 7, 2, 18, 31, 2, 18, 7, 2, 18, 31, 2, 9, 7, 2, 9, 32, 2, 9, 7, 2, 9, 32, 2, 16, 7, 2, 16, 13, 2, 16, 7, 2, 16, 13, 2, 8, 7, 2, 8, 36, 2, 8, 7, 2, 8, 36, 2, 9, 0, 19, 1], [7, 2, 18, 31, 2, 18, 7, 2, 18, 31, 2, 9, 7, 2, 9, 32, 2, 9, 7, 2, 9, 32, 2, 16, 7, 2, 16, 13, 2, 16, 7, 2, 16, 13, 2, 8, 7, 2, 8, 36, 2, 8, 7, 2, 8, 36, 2, 18, 7, 2, 18, 31, 2, 18, 7, 2, 18, 31, 2, 9, 7, 2, 9, 32, 2, 9, 7, 2, 9, 32, 2, 16, 7, 2, 16, 13, 2, 16, 7, 2, 16, 13, 2, 8, 7, 2, 8, 36, 2, 8, 7, 2, 8, 36, 2, 9, 0, 19, 1, 8]]
1263


In [12]:
from keras.utils import to_categorical
X_one_hot = to_categorical(X_train)
Y_one_hot = to_categorical(Y_train)

In [16]:
print("X_one_hot shape is:",X_one_hot.shape)
print("Y_one_hot shape is:",Y_one_hot.shape)

X_one_hot shape is: (1263, 100, 77)
Y_one_hot shape is: (1263, 77)


In [17]:
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Input, Model
from keras.layers import  Dropout, Dense,LSTM 
from keras.optimizers import Adam
from keras.utils import plot_model
# X_one_hot.shape[1:] = (100, 308)
input_tensor = Input(shape=X_one_hot.shape[1:])
lstm = LSTM(512,return_sequences=True)(input_tensor)
dropout = Dropout(0.3)(lstm)

lstm = LSTM(256)(dropout)
dropout = Dropout(0.3)(lstm)
# Y_one_hot.shape[-1] = 308
dense = Dense(Y_one_hot.shape[-1], activation='softmax')(dropout)

model = Model(inputs=input_tensor, outputs=dense)
# 画图
# plot_model(model, to_file='model.png', show_shapes=True, expand_nested=True, dpi=500)
optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 77)]         0         
_________________________________________________________________
lstm (LSTM)                  (None, 100, 512)          1208320   
_________________________________________________________________
dropout (Dropout)            (None, 100, 512)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 77)                19789     
Total params: 2,015,565
Trainable params: 2,015,565
Non-trainable params: 0
____________________________________________

In [18]:
filepath = "./{epoch}--weights{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',  # 监控的对象是loss
    verbose=0,
    save_best_only=True,
    mode='min'  # 如果监控对象是val_acc则取max，是loss则取min
)
callbacks_list = [checkpoint]
model.fit(X_one_hot, Y_one_hot, epochs=100, batch_size=100,callbacks=callbacks_list)

Epoch 1/100
13/13 [==============================] - 11s 869ms/step - loss: 3.9163 - accuracy: 0.1409
Epoch 2/100
10/13 [======================>.......] - ETA: 2s - loss: 3.3681 - accuracy: 0.1940

KeyboardInterrupt: 